# Väderdata 2
Denna uppgift är en vidareutveckling på väderdata. Vi rekommenderar starkt att du gör den först. Du kan hitta den [här](https://github.com/lunduniversity/schoolprog-satellite/tree/master/exercises/weatherdata). Vi kommer använda samma data i denna uppgiften men göra den mer lättillgänglig genom att göra en interaktiv graf med hjälp av biblioteket [bokeh](https://bokeh.pydata.org/en/latest/).

Börja med att köra följande kod för att importera rätt bibliotek:

In [0]:
from ipywidgets import interact
import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure

output_notebook()

Koden nedan laddar ned datan vi kommer använda och sparar den i `data`. Funktionerna är samma som i den första väderdatauppgiften. Kör koden för att läsa in datan.

In [0]:
!wget https://github.com/lunduniversity/schoolprog-satellite-data/raw/master/smhi/1961.all.ssv.gz --no-verbose
import gzip
import matplotlib.pyplot as plt


def get_station_data():
    result = {}
    with gzip.open('1961.all.ssv.gz', 'rt') as f:
        raw_data = f.read().split('\n\n')
        for s in raw_data:
            station_data = s.split('\n')
            name = ' '.join(station_data[0].split()[1:])
            data = []
            for entry in station_data[1:]:
                y, m, d, t = entry.split()
                data.append((int(y), int(m), int(d), float(t)))
            if(len(name) > 0):
                result[name] = data
    return result


def plot(x=None, y=None, fname="plot.png"):
    if x: 
        plt.plot(x, y)
    else:
        plt.plot(y)
    plt.savefig(fname)
    plt.show()
    
def data_by_year(year, city_data):
    result = []
    for datum in city_data:
        if datum[0] == year:
            result.append(datum[3])
    return result

data = get_station_data()

## 1. Plotta med `bokeh`

Nu ska vi börja med `bokeh`. I `bokeh` plottar man saker i en `figure`. Man skapar en `figure` på följande vis:

In [0]:
plot = figure(title="Temperatur för ett år", plot_height=300, plot_width=600, y_range=(-30, 30))

**Uppdrag:** Vad gör de olika parametrarna? Försök lista ut bara utifrån namnen.

<details>
<summary markdown="span">
Svar
</summary>
<p>
    
- <code>title</code> sätter titeln för plotten.
    
- <code>plot_height</code> bestämmer höjden på plotten.

- <code>plot_width</code> bestämmer bredden på plotten.

- <code>y_range</code> avgör vilka y-värden som ska vara med i plotten.

</p>
</details>

I en `figure` i bokeh kan man lägga till exempelvis linjer, vilket vi ska göra nu. Men först behöver vi någon data att plotta med linjen. 

**Uppdrag:** skapa en lista `y` som innehåller temperaturerna från Lund 2015 samt en lista `x` som innehåller talen 1, 2 till och med 365. Detta ska representera dagarna. 



<details>
<summary markdown="span">
Tips
</summary>
<p>
    Använd funktionen <code>data_by_year()</code> du skrev i den förra väderuppgiften. 
</p>
</details>

<details><summary markdown="span">Lösning</summary>
<p>
<pre><code>y = data_by_year(2015, data["Lund"])
x = [i+1 for i in range(365)]
</code></pre></p>
</details>

Nu när du skapat listorna är det dags att plottad dem. 

**Uppdrag:** Kör koden nedan för att först lägga till en linje som du kallar `my_line` och visa plotten med hjälp av `show(plot)`.

In [0]:
my_line = plot.line(x, y)
show(plot)

Som du ser är denna plotten inte helt lik den du känner igen sen tidigare. Innan använde vi `matplotlib.pyplot` och nu använder vi `bokeh`. Du kan direkt se hur `bokeh` är mer interaktivt genom att du kan välja olika verktyg på sidan om plotten som låter dig zooma och dra runt plotten. Men vi ska göra plotten ännu mer interaktiv!

## 2. Interaktiv plot med `bokeh`

**Uppdrag:** Skriv en funktion `formatted_data_by_year(year, city)` som tar in ett år och en stad. Funktionen ska returnera nästan samma sak som `data_by_year()` men den ska fixa till datan så att vi alltid har 365 dagar per år. Detta är eftersom vissa städer saknar data för exempelvis de sista månaderna på året, och ibland är det skottår.



In [0]:
def formatted_data_by_year(year, city):
    # Din kod här..

<details>
<summary markdown="span">
Tips
</summary>
<p>
Undvik att göra någon komplicerat. Du kan exempelvis göra följande: om det finns för många värden tar du bort det sista värdet tills du har exakt 365. Om du tvärtemot har för få värden kan du lägga till det sista värdet till listan tills du har exakt 365. 
</p>
</details>

<details><summary markdown="span">Lösning</summary>
<p>
<pre><code>def formatted_data_by_year(year, city):
    res = data_by_year(year, data[city])
    while (len(res) > 365):
        res = res[:-1]
    while (len(res) < 365):
        res.append(res[-1])
    return res
</code></pre></p>
</details>


Nu ska vi börja använda `ipywidgets` och dess funktion `interact`.

**Uppdrag:** Kör följande kod. Vad händer? Varför? Försök förstå vad de olika delarna gör. 

In [0]:
def update(city, year):
    my_line.data_source.data['y'] = formatted_data_by_year(year, city)
    push_notebook(handle=my_handle)
    
my_handle=show(plot, notebook_handle=True)
interact(update, city=["Lund", "Stockholm"], year=(1965, 2016, 1))

<details>
<summary markdown="span">
Svar
</summary>
<p>

- <code>update(city, year)</code> är en funktion som ska uppdatera grafen varje gång något ändras från inställningarna. 
- <code>my_line.data_source.data['y'] = formatted_data_by_year(year, city)</code> ändrar linjens y-värden till värdena som returneras av <code>formatted_data_by_year()</code>. 
- <code>push_notebook(handle=my_handle)</code> uppdaterar grafen som har <code>my_handle</code> som handle. 
- <code>my_handle=show(plot, notebook_handle=True)</code> gör så att grafen får en handle som vi kallar <code>my_handle</code>. 
- <code>interact()</code> är funktionen som ger listan av alternativ och en slider där man kan ange vilket år man vill ha datan ifrån. <code>interact()</code> tar först en <code>update</code>-funktion som anropas varje gång värdena uppdateras. Sedan måste man för varje parameter till <code>update</code> ange hur man ska läsa in dem. Anger man en lista av strängar får man en flervalslista och anger man en tuple får man en slider. De två första värdena i tupeln anger start och slutvärde. Det tredje anger hur små steg man kan ta. 
</p>
</details>

Nu vill vi kunna se fler städer. 

**Uppdrag:** Ändra i koden ovan så att vi får alla städer vi har i flervalslistan. 


<details><summary markdown="span">Lösning</summary>
<p>
<pre><code>interact(update, city=list(data.keys()), year=(1965, 2016,1))
</code></pre></p>
</details>

## 3. Ackumulerad temperatur

En ackumulerad tempertur innebär att man summerar alla de tidigare temperaturerna. Om det exempelvis har varit 20 grader flera dagar i streck blir den ackumulerade temperaturen dag ett 20 grader, dag två 40 grader, dag tre 60 grader osv. Vi tänke nu kolla på hur den ackulmulerade temperaturen ser ut för ett år i de olika städerna vi kollat på innan. 

För att göra detta behöver vi skriva en funktion som hela tiden läger ihop de tidigare värdena med dagens värde. Vi kan utgå från vår funktion `formatted_data_by_year()`. 

**Uppdrag:** Skriv funktionen `formatted_data_by_year_ack(year,city)`.


In [0]:
def formatted_data_by_year_ack(year, city):
  # Din kod här..

<details>
<summary markdown="span">
Tips
</summary>
<p>
  Vi kan exempelvis loopa genom listan från <code>formatted_data_by_year</code> och för varje element lägga till värdet av det tidigare elementet (som då kommer vara det ackumulerade värdet för det elementet).
</p>
</details>

<details>
<summary markdown="span">
Lösning
</summary>
<p>
<pre>def formatted_data_by_year_ack(year, city):
    res = formatted_data_by_year(year, city)
    for i in range(1,365):
        res[i] = res[i] + res[i-1]
    return res</pre>
</p>
</details>


Vi kan nu plotta våra ackumulerade värden på samma sätt som innan. Koden nedan är i princip samma som du skrev i del 2. Vi väljer denna gång att göra ett stapeldiagram istället för en linje. Kör koden nedan för att se vad som händer. 

In [0]:
plot2 = figure(title="Ackumulerad temperatur för ett år", plot_height=300, plot_width=600, y_range=(-2000, 4000))
y = formatted_data_by_year_ack(2015, "Lund")
x = [i+1 for i in range(365)]
my_vbar = plot2.vbar(x, top=y, width = 0.5)

def update_ack(city, year):
    my_vbar.data_source.data['top'] = formatted_data_by_year_ack(year, city)
    push_notebook(handle=my_handle2)
    
my_handle2=show(plot2, notebook_handle=True)
interact(update_ack, city=list(data.keys()), year=(1965, 2016, 1))

**Uppdrag:** Kolla på lite olika städer. Vad innebär det om den ackumulerade temperaturen är under noll i slutet av året?

<details>
<summary markdown="span">
Svar
</summary>
<p>
Det innebär att medeltemperaturen för hela året är negativ. Vi kan bestämma medeltemperaturen genom att ta det sista värdet, det totala ackumulerade värdet för året, och dela det på 365.
</p>
</details>

## Fortsättningsuppgifter
- Gör om fler plottar från förra uppgiften och gör dem interaktiva. 
- Ändra så att man i `update`-även uppdaterar x-värdena för att slippa använda `formated_data_by_year` och istället bara använda `data_by_year`.